# Querying On-Disk Tables

> Objectives:
> * Compare queries of tabular data for **on-disk** containers
> * Compare sizes and times for those

In [ ]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

## Load movielens datasets in pandas DataFrames

Let's start loadind dataset in pandas so that we can create persistent versions more easily later on.

In [ ]:
import os
dset = 'movielens-1m'
fdata = os.path.join(dset, 'ratings.dat.gz')
fitem = os.path.join(dset, 'movies.dat')

In [ ]:
# Import CSV files via pandas
import pandas as pd
# pass in column names for each CSV
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(fdata, sep=';', names=r_cols, compression='gzip')

m_cols = ['movie_id', 'title', 'genres']
movies = pd.read_csv(fitem, sep=';', names=m_cols,
                     dtype={'title': object, 'genres': object})

In [ ]:
# Store movies and ratings in 2 separate tables in SQLite
sqlite_norm = "movielens-norm.sqlite"
if os.path.exists(sqlite_norm):
    os.unlink(sqlite_norm)
import sqlite3
conn = sqlite3.connect(sqlite_norm)
conn.text_factory = str   # Shut up problems with Unicode
ratings.to_sql("ratings", conn)
movies.to_sql("movies", conn)
conn.close()

In [ ]:
# Create one merged DataFrame
lens = pd.merge(movies, ratings)

In [ ]:
lens.info()

In [ ]:
# Store lens denormalized in 1 table in SQLite
sqlite_denorm = "movielens-denorm.sqlite"
if os.path.exists(sqlite_denorm):
    os.unlink(sqlite_denorm)
conn_denorm = sqlite3.connect(sqlite_denorm)
conn_denorm.text_factory = str   # Shut up problems with Unicode
lens.to_sql("lens", conn_denorm)
conn_denorm.close()

In [ ]:
# Get rid of unnecessary containers to save memory
del movies, ratings

In [ ]:
%time result = lens.query("(title == 'Tom and Huck (1995)') & (rating == 5)")['user_id']
result

In [ ]:
# keep the run-time for reference
qtime_pandas_mem = mw.measurements.time_delta
qtime_pandas_mem

## bcolz

`bcolz` cannot only deal with data in-memory, but also on-disk using exactly the same API.  Let's try that.

In [ ]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'lz4hc'
bcolz.defaults.cparams['clevel'] = 6
bcolz.defaults.cparams['shuffle'] = bcolz.BITSHUFFLE
bcolz.set_nthreads(4)

In [ ]:
# Import a pandas DataFrame in a ctable on-disk
bcolz_dir = "movielens-denorm.bcolz"
if os.path.exists(bcolz_dir):
    import shutil
    shutil.rmtree(bcolz_dir)
zlens = bcolz.ctable.fromdataframe(lens, rootdir=bcolz_dir)

In [ ]:
# Workaround to use bytes instead of Unicode (not supported yet in numexpr queries)
title = zlens['title'][:].astype('S82')
zlens.delcol('title')
zlens.addcol(title, 'title')

In [ ]:
# Workaround to use bytes instead of Unicode (not supported yet in numexpr queries)
genres = zlens['genres'][:].astype('S47')
zlens.delcol('genres')
zlens.addcol(genres, 'genres')

In [ ]:
%time result = zlens["(title == b'Tom and Huck (1995)') & (rating == 5)"]['user_id']
result

In [ ]:
# keep the run-time for reference
qtime_bcolz = mw.measurements.time_delta
# ...and print the result (for reference too)
result

In [ ]:
# One can optimize the query more
%time result = [r.user_id for r in zlens.where("(title == b'Tom and Huck (1995)') & (rating == 5)", outcols=['user_id'])]

In [ ]:
# keep the run-time for reference
qtime_bcolz_opt = mw.measurements.time_delta
# ...and print the result (for reference too)
result

Let's see the space consumed on-disk:

In [ ]:
!du -sh $bcolz_dir

and let's compare it with the internal estimated size:

In [ ]:
zlens

So, the internal estimated size is a bit higher than the actual size on-disk, which means that bcolz offers a conservative guess in general.

But there are other ways to store compressed tables.  Let's visit PyTables and see how it performs.

## PyTables

In [ ]:
h5denorm = "movielens-denorm.h5"
if os.path.exists(h5denorm):
    os.unlink(h5denorm)
zlens.tohdf5(h5denorm, nodepath='/h5lens')

In [ ]:
import tables
h5file = tables.open_file(h5denorm)
h5lens = h5file.root.h5lens

In [ ]:
# Let's have a look at the HDF5 table
h5lens

In [ ]:
%time result = [r['user_id'] for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == 5)")]

In [ ]:
# keep the run-time for reference
qtime_pytables = mw.measurements.time_delta
# ...and print the result (for reference too)
result

So, it seems that PyTables is ~2x slower than bcolz.  What about the size of the HDF5 file?

In [ ]:
h5file.close()
!ls -lh $h5denorm

6.0 MB is approximately the same than the 6.0 MB that used bcolz (which is expected because both are using LZ4HC as the compressor).

## Indexing (PyTables)

PyTables can index tables for improving query speed.  Let's try that out:

In [ ]:
## Copy the original PyTables table into another file
import shutil
h5idx = "movielens-indexed.h5"
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5denorm, h5idx)

In [ ]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")

In [ ]:
# Create an index for the 'title' column
h5lens = h5i.root.h5lens
h5lens.cols.title.create_csindex()

In [ ]:
# ...and redo the query...
t = %timeit -r1 -n1 -o result = [r['user_id'] for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == 5)")]

In [ ]:
# keep the run-time for reference
qtime_pytables_index1 = t.best
# ...and print the result (for reference too)
result

So, by indexing one column we have got more than 20x of acceleration wrt PyTables.  What happens if we index the 'rating' column?

In [ ]:
h5lens.cols.rating.create_csindex()

In [ ]:
t = %timeit -r1 -n1 -o result = [r['user_id'] for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == 5)")]

In [ ]:
# keep the run-time for reference
qtime_pytables_index2 = t.best
# ...and print the result (for reference too)
result

Ok, that's another ~3x or ~4x additional acceleration, and the best time that we ever reached for this query.

In [ ]:
h5i.close()

## Querying on-disk data with SQLite (relational database)

In [ ]:
# Execute the query with the de-normalized SQLite database
conn = sqlite3.connect(sqlite_denorm)
c = conn.cursor()

In [ ]:
q = "select user_id from lens where title == 'Tom and Huck (1995)' and rating == 5"
%time result = [r for r in c.execute(q)]

In [ ]:
# keep the run-time for reference
qtime_sqlite_denorm = mw.measurements.time_delta
# ...and print the result
result

In [ ]:
# Execute the query with the normalized SQLite database
conn = sqlite3.connect(sqlite_norm)
c = conn.cursor()

In [ ]:
q = """
select ratings.user_id from movies 
INNER JOIN ratings ON movies.movie_id = ratings.movie_id
where movies.title == 'Tom and Huck (1995)' and ratings.rating == 5
"""
%time result = [r for r in c.execute(q)]

In [ ]:
# keep the run-time for reference
qtime_sqlite_norm = mw.measurements.time_delta
# ...and print the result
result

So in general, we see that it is much faster to query tables in denormalized form, although they take much more storage:

In [ ]:
!ls -lh $sqlite_norm $sqlite_denorm

In some way, storing tables in normalized form is a kind of compression, but that comes to the cost of using more time to process queries.  But indexing is the strongest point of relational databases, so let's see how much it can accelerate queries:

## Indexing (SQLite)

In [ ]:
sqlite_idx = "movielens-indexed.sqlite"
if os.path.exists(sqlite_idx):
    os.unlink(sqlite_idx)
shutil.copyfile(sqlite_denorm, sqlite_idx)
conn_idx = sqlite3.connect(sqlite_idx)

In [ ]:
c = conn_idx.cursor()
c.execute("CREATE INDEX index_title ON lens (title)")
conn_idx.commit()

In [ ]:
q = "select user_id from lens where title == 'Tom and Huck (1995)' and rating == 5"
t = %timeit -r1 -n1 -o result = [r for r in c.execute(q)]
# keep the run-time for reference
qtime_sqlite_index1 = t.best

In [ ]:
# ...and print the result (for reference too)
result

Wow, with quite less than 1 ms we are getting the best figure so far, and faster than the best figure with indexed PyTables.

## Exercise

What will happen if we index the rating column?  We should get better speed, right?  Try it out and report the improvement that you are seeing.

### Solution

## Final performance comparison

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# Query times
labels = ["pandas (in-memory)", "bcolz", "PyTables", "SQLite denorm", "SQLite norm", "PyTables (indexed)", "SQLite (indexed)"]
df = pd.DataFrame({'time (sec)': [qtime_pandas_mem, qtime_bcolz_opt, qtime_pytables, qtime_sqlite_denorm, qtime_sqlite_norm, qtime_pytables_index2, qtime_sqlite_index1]}, index=labels)
df.plot(kind='barh', figsize=(12,5), fontsize=16, title="Query times for MovieLens 1m")

## Final size comparison

In [ ]:
!du -sh movielens* | sort -n

In [ ]:
# Container sizes
df = pd.DataFrame({'size (MB)': [160, 6.0, 6.0, 78, 37, 9.9, 119]}, index=labels)
df.plot(kind='barh', figsize=(12,5), fontsize=16, title="Container sizes for MovieLens 1m")

# Final Overview

Fortunately, there are many different solutions for storing large datasets both in-memory and on-disk.  Here it is a summary of the ones that we have seen in this tutorial:

* In-core
  * Python lists and dictionaries: Included in Python.  Very flexible.  Not efficient for large datasets.
  * NumPy: Good for large amounts of data and with a lot of functionality.  Does not support compression.
  * Pandas: Good for large amounts of data and with a lot of functionality.  Does not support compression.
  * bcolz: Good for large amounts of data, but reduced functionality.  Supports compression.
  
* Out-of-core
  * bcolz: Good for large datasets.  Supports compression, but not indexing.  Medium query speeds.
  * SQLite: Can be used for large datasets, but requires lots of storage.  Supports indexing.  Excellent query speeds.
  * PyTables: Good for large datasets.  Supports indexing and compression.  Good query speeds.

As you see, every solution has strengths and weaknesses, with wild variations in resource consumptions, so a wise thing to do is to know them better and try to apply the best candidate to your scenario.

As a final note, this series of tutorials is work in progress, so please contact the author in case you see some inacuracy or just want to send feedback.

**Hope you enjoyed the ride!**